In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from tracking_grants.utils.helpers import load_articles, load_metrics, load_awards, load_grants

In [4]:
articles = load_articles()
metrics = load_metrics()
awards = load_awards()
grants = load_grants()

# Tracking Grants

29.07.2020, ScholCommLab

## Part 0 - Goals/Overview

### What we hope this data can do (and what it cannot)

- Enhance the available data regarding these programs with other sources
- Provide an high-level overview of the outputs of each funding program
- Give a sense of future areas of exploration

### What we are not doing

- Not doing a detailed assessment of grants, papers, or programs
- Not evaluating the quality of the work, efficiency of the programs, of measuring if they are meeting their stated objectives. 
- Citations ≠ quality, Altmetrics ≠ public value, News ≠ saliency

## Part 1 - Data Gathering and Enhancement

### Overview of the data pipeline

![](tracking_grants_flowchart.png)

### Overall descriptives

- Which research programs are we using?
- How many articles?
- How many awards?
- Distribution of years published in?

### Coverage on the publication level

- How many DOIs could we identify and match: Venn Diagram of:
    - All articles
    - Matched DOI
- 2nd Venn Diagrams of DOIs
    - Matched with PMID
    - Matched in WoS
    - Matched in Altmetric.com

### Clinical Trials

- How many articles (with DOIs) had clinical trials?
- How many grants had clinical trials?
- Distribution of phases (table?)

## Part 2 - Data on the grant level

After collecting article-level data we can offer insights on the grant level by aggregating by the IDs.

### Slide - Overview of grants

- How many per research program?
- Average size of awards (publications, award sizes)

### Slide - Measuring grant outputs

Boxplots for various metrics: citations, twitter, news, ...

## Part 3 - Not all grants are the same

### Overview of grants
- How many per research program?
- Average size/distribution of awards
    - Number of Publications
    - Award sizes

### Award Types
- Break down # of Publications and $ by award type

## Part 4 - Not all grants are the same

### Measuring grant outputs

- Boxplots for various metrics: citations, twitter, news, ..

### Pairplot on the grant level (3x3 plots)

- Color by award type?
- Size of bubble by funding amount
- 3 plots: Citations, Twitter, News, (Clinical trials?)

### Pairplots: 

- Bubbles are grants
    - \$ vs # paper
    - \$ vs # citation
    - \$ vs # tweet 

## Part 5 - Limitations

### Data quality and completeness

- We are not fetching papers, only using data provided
- We are matching citation strings to Crossref database: there’s an error rate in this matching
- Citations from WoS only count citing papers from other WoS papers
- Altmetric and Unpaywall are also known to have some errors (as with any database drawn from searching the Web).